# Homework 2

In [2]:
##### IMPORTS
import numpy as np
import math

In [11]:
def RegressionAtHome(X_train, Y_train, x_pred, tau):
    
    weights = []

    # class_weight
    # {class_label : weight}
    
    # what is m in the cost function???
    # use class_weights to incorporate weights into class function
    # but what is m??? and will this actually work?
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)
    
    
    # modify class_weight = weights
    
    pass #FIXME


# Gradient is first partial derivatives of the cost function
# Hessian Matrix is second partial derivatives of the cost function
# Newton's method uses gradient and hessian for,,,, something


# J function -- this is what we're trying to optimize
# do we actually need this?
def J(lamb, theta, w_i, y_i, x_i, m):  # can't call it lambda bc python
    sum = 0
    for j in range(m):
        sum += w_i * (y_i * math.log(h(theta, x_i)) + (1 - y_i) * math.log(1 - h(theta, x_i)))
    return ((-1 * lamb * magnitude(theta)**2) / 2) + sum

# calculate the sigmoid function for x_i
def h(theta, x_i):
    # sigmoid function
    x_i = np.array(x_i)
    t = np.matmul(x_i.T, theta)  # if we're using the transpose of x_i, is this basically just a dot product? does this give a scalar?
    val = 1 / (1 - math.exp(-1 * t))
    # return 1 or 0 based on calculated value?
    if val > 0.5:
        return 1
    else:
        return 0

# calculate gradient of J (w.r.t theta)
def gradient_J(X, w_i, y_i, x_i, lamb, theta):
    gradient = np.matmul(X.T, w_i * (y_i - h(theta, x_i))) - lamb * theta
    return gradient


# Hessian Matrix???
def hessian_matrix(X, D, lamb):
    hessian = np.matmul(np.matmul(X.T, D), X) - lamb * np.shape(D)[0]
    return hessian


# Diagonal Matrix????????
def diagonal(w_i, x_i, theta):
    D_ii = -1 * w_i * h(x_i, theta) * (1 - h(x_i, theta))
    return D_ii



# calculate the magnitude of a vector
def magnitude(vector):
    mag = np.linalg.norm(vector)
    return mag

# calculate the weight based on point x_q
def calc_weight(x_q, x_i, tau):
    x_q = np.array(x_q)
    x_i = np.array(x_i)
    difference = x_q - x_i
    w_i = math.exp((magnitude(difference)**2) / (2 * tau**2))
    return w_i


##### PRINTS DOWN HERE

1.0490821765437979


## Problem 2

1. Soft-Margin Linear SVM, C=.02 corresponds to #4. This is a linear model with larger margins, so the C value is lower.
2. Soft-Margin Linear SVM, C=20 corresponds to #3. This is a linear model with smaller margins, so the C value is higher.
3.